# LPjuLinearModel

In [4]:
using LinearAlgebra
using SparseArrays
using Random
using ArgParse
using DataStructures
using Test

# JuMP
using JuMP
using MathOptInterface
const MOI = MathOptInterface

MathOptInterface

In [13]:
include("../src/problems/lp_problem_structs.jl")
include("../src/file_formats/lp_file_formater.jl")
include("../src/preprocess/lp_presolve.jl")
include("../src/preprocess/lp_standard_form_converter.jl")
include("../src/lp_revised_simplex.jl")

revised_simplex

In [6]:
mps_folder_path = "../check/problems/mps_files/"

"../check/problems/mps_files/"

In [7]:
mps_string = read_file_to_string(mps_folder_path * "blend.mps")
println(mps_string)

NAME          BLEND

OBJSENSE
 MAX
 
ROWS
 N  PROF
 L  VVEG
 L  NVEG
 L  UHRD
 G  LHRD
 E  CONT
COLUMNS
    VEG01     PROF      -110.0    VVEG         1.0
    VEG01     UHRD         8.8    LHRD         8.8
    VEG01     CONT         1.0
    VEG02     PROF      -120.0    VVEG         1.0
    VEG02     UHRD         6.1    LHRD         6.1
    VEG02     CONT         1.0
    OIL01     PROF      -130.0    NVEG         1.0
    OIL01     UHRD         2.0    LHRD         2.0
    OIL01     CONT         1.0
    OIL02     PROF      -110.0    NVEG         1.0
    OIL02     UHRD         4.2    LHRD         4.2
    OIL02     CONT         1.0
    OIL03     PROF      -115.0    NVEG         1.0
    OIL03     UHRD         5.0    LHRD         5.0
    OIL03     CONT         1.0
    PROD      PROF       150.0    UHRD        -6.0
    PROD      LHRD        -3.0    CONT        -1.0
RHS
    RHS       VVEG       200.0
    RHS       NVEG       250.0
    RHS       UHRD         0.0
    RHS       LHRD         0.0
 

In [24]:
lp = read_mps(mps_folder_path * "ex_9-7.mps")

LPProblem(false, [4.0, 3.0, 1.0, 7.0, 6.0], sparse([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5], [1.0, 2.0, -3.0, 2.0, -1.0, 2.0, 3.0, 2.0, 1.0, 1.0, 2.0, -1.0, -3.0, 1.0, 2.0], 3, 5), [9.0, 10.0, 11.0], ['L', 'L', 'L'], [0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf], ["X1", "X2", "X3", "X4", "X5"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous])

In [16]:
pre_lp = presolve_lp(lp)

PreprocessedLPProblem(LPProblem(false, [4.0, 3.0, 1.0, 7.0, 6.0], sparse([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5], [1.0, 2.0, -3.0, 2.0, -1.0, 2.0, 3.0, 2.0, 1.0, 1.0, 2.0, -1.0, -3.0, 1.0, 2.0], 3, 5), [9.0, 10.0, 11.0], ['L', 'L', 'L'], [0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf], ["X1", "X2", "X3", "X4", "X5"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous]), LPProblem(false, [4.0, 3.0, 1.0, 7.0, 6.0], sparse([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5], [1.0, 2.0, -3.0, 2.0, -1.0, 2.0, 3.0, 2.0, 1.0, 1.0, 2.0, -1.0, -3.0, 1.0, 2.0], 3, 5), [9.0, 10.0, 11.0], ['L', 'L', 'L'], [0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf], ["X1", "X2", "X3", "X4", "X5"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous]), Int64[], Int64[], Dict{Int64, Tuple{Int64, Float64}}(), Dict{String, Float64}(), Float64[], Float64[], false)

In [17]:
revised_simplex(pre_lp)

(Dict("s_1" => 0.0, "X3" => 0.0, "X4" => 0.0, "s_3" => 0.0, "X5" => 6.000000000000001, "s_2" => 0.0, "X2" => 10.000000000000002, "X1" => 7.000000000000001), 94.00000000000003)

In [20]:
using BenchmarkTools
using Profile
using ProfileView

In [25]:
@benchmark revised_simplex(pre_lp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  21.208 μs …   6.579 ms  ┊ GC (min … max): 0.00% … 25.29%
 Time  (median):     23.459 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.088 μs ± 131.360 μs  ┊ GC (mean ± σ):  5.23% ±  1.21%

      ▁▁▁▃▅▆▆███▇▆▆▃▂▁                                          
  ▂▃▄▇██████████████████▆▅▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂ ▄
  21.2 μs         Histogram: frequency by time         31.2 μs <

 Memory estimate: 90.89 KiB, allocs estimate: 515.

In [22]:
ProfileView.@profview revised_simplex(pre_lp)

Gtk4.GtkWindowLeaf(accessible-role=GTK_ACCESSIBLE_ROLE_APPLICATION, name="", parent, root, width-request=-1, height-request=-1, visible=true, sensitive=true, can-focus=true, has-focus=false, can-target=true, focus-on-click=true, focusable=false, has-default=false, receives-default=false, cursor, has-tooltip=false, tooltip-markup=nothing, tooltip-text=nothing, opacity=1.000000, overflow=GTK_OVERFLOW_HIDDEN, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, hexpand=false, vexpand=false, hexpand-set=false, vexpand-set=false, scale-factor=2, css-name="window", css-classes, layout-manager, title="Profile  -  14:33:14", resizable=true, modal=false, default-width=800, default-height=600, destroy-with-parent=false, hide-on-close=false, icon-name=nothing, display, decorated=true, deletable=true, transient-for, application, default-widget, focus-widget, child, titlebar, handle-menubar-accel=true, is-active=false, suspended=false, startup-i

In [23]:
ProfileView.print()

In [26]:
using PProf

┌ Warning: Module FlameGraphs with build ID fafbfcfd-1e23-1959-0006-45e6d7ea98f8 is missing from the cache.
│ This may mean FlameGraphs [08572546-2f56-4bcf-ba4e-bab62c3a3f89] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948


In [32]:
Profile.Allocs.@profile revised_simplex(pre_lp)
PProf.Allocs.pprof()

Analyzing 78 allocation samples... 100%|█████████████████| Time: 0:00:00


"alloc-profile.pb.gz"

In [35]:
PProf.Allocs.pprof()


"alloc-profile.pb.gz"

include("src/problems/lp_problem_structs.jl")
include("src/file_formats/lp_file_formater.jl")
include("src/preprocess/lp_presolve.jl")
include("src/preprocess/lp_standard_form_converter.jl")
include("src/solvers/lp_revised_simplex.jl")


lp = read_mps(mps_folder_path * "ex_9-7.mps")

 pre_lp = presolve_lp(lp)

Profile.Allocs.@profile for _ in 1:10
    revised_simplex(pre_lp)
end

PProf.Allocs.pprof()